# California Housing Prices Data Set

Create a regression model to predict house prices. See https://www.kaggle.com/camnugent/california-housing-prices for data understanding



## Import packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

import seaborn as sns
sns.set()

## Load and inspect data set

In [ ]:
# Fetch the file
my_file = project.get_file("housing.csv")

# Read the CSV data file from the object storage into a pandas DataFrame
my_file.seek(0)
original_data = pd.read_csv(my_file)

original_data.head()

In [ ]:
original_data.describe(include='all') # descriptive statistics for all columns

In [ ]:
original_data.isnull().sum() # check for null values

In [ ]:
original_data[original_data.duplicated(keep=False)] # check for duplicate rows

There are no duplicates but missing values for "total_bedrooms". Decide what to do with these null values: 

In [ ]:
data_wo_null = # your code
data_wo_null.isnull().sum() # check

In [ ]:
# Musterlösung 
data_wo_null = original_data.dropna(axis=0)
data_wo_null.isnull().sum()

## Select predictors

Create a correlation map:

In [ ]:
# your code

In [ ]:
# Musterlösung
f,ax=plt.subplots(figsize = (18,18))
sns.heatmap(data_wo_null.corr(),annot= True,linewidths=0.5,fmt = ".1f",ax=ax)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.title('Correlation Map')
plt.show()

Remove redundant features: 

In [ ]:
data_reduced_features = data_wo_null[['<your feature 1>', '<your feature 2>','...']]
data_reduced_features.head()

In [ ]:
# Musterlösung
data_reduced_features = data_wo_null[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'median_income', 
                                      'median_house_value', 'ocean_proximity']] 
                                        # either keep longitude and latitude or remove both
data_reduced_features.head()

## Remove outliers
The next step is to detect outliers and handle them:

In [ ]:
data_reduced_features.hist(figsize=(25,25), bins=50)

In [ ]:
q = # your code

data_reduced_features_2 = # your code

In [ ]:
# Musterlösung, total_rooms only, more outliers might be removed! 
q = data_reduced_features['total_rooms'].quantile(0.99)
data_reduced_features_2 = data_reduced_features[data_reduced_features['total_rooms']<q]

## Prepare data for modeling

Get dummies since there is a categorical feature:

In [ ]:
dummies = # your code
dummies.head()

In [ ]:
# Musterlösung
dummies = pd.get_dummies(data_reduced_features_2, drop_first=True)
dummies.head()

Set X and y (predictors and target) according to your dataframe:

In [ ]:
target = dummies['<your target column>']
predictors = # your code

In [ ]:
# Musterlösung
target = dummies['median_house_value']
predictors = dummies.drop(['median_house_value'], axis=1)

Split data into training and test sets: 

In [ ]:
X_train, X_test, y_train, y_test = # your code

In [ ]:
# Musterlösung
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.2, random_state=123) # 80-20 split into training and test data

Use StandardScaler to scale your predictors:

In [ ]:
scaler = # your code 
# your code

X_train = # your code
X_test = # your code

In [ ]:
# Musterlösung

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Regression model and evaluation

Create a linear regression model: 

In [ ]:
# your code

In [ ]:
# Musterlösung

reg = LinearRegression()
reg.fit(X_train,y_train)


In [ ]:
print('training performance')
print(reg.score(X_train,y_train))
print('test performance')
print(reg.score(X_test,y_test))

In [ ]:
y_pred = reg.predict(X_test)
test = pd.DataFrame({'Predicted':y_pred,'Actual':y_test})
fig= plt.figure(figsize=(16,8))
test = test.reset_index()
test = test.drop(['index'],axis=1)
plt.plot(test[:50])
plt.legend(['Actual','Predicted'])
sns.jointplot(x='Actual',y='Predicted',data=test,kind='reg',);

Interpret the result and feel free to try out further analyses.

# Musterlösung
Training and test performance similar (= no overfitting)
simple linear regression model seems to be ok
cf. https://people.duke.edu/~rnau/rsquared.htm if you are interested in learning more about evaluating regression models.

# Deployment

Deploy the linear regression model via the _Watson Machine Learning_ (WML) service on IBM Cloud. Please refer to the documentation for more details about the [watson-machine-learning-client](https://pypi.org/project/watson-machine-learning-client/) or the [REST API](https://watson-ml-api.mybluemix.net/).

In [ ]:
# import the Watson Machine Learning Python client library
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
# fill in your credentials
wml_credentials = {
    # your code
}

In [ ]:
# instantiate the client
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# review stored artifacts
wml_client.repository.list()

In [ ]:
# delete artifacts that are no longer needed (note that the number of stored artifacts in the free version is limited to 5)
wml_client.repository.delete("...GUID...")

In [ ]:
# fill in metadata for your deployment
metadata = {
        wml_client.repository.ModelMetaNames.NAME: 'Housing Prices Deployment',
        wml_client.repository.ModelMetaNames.DESCRIPTION: 'Test deployment for the california housing prices data set.',
        wml_client.repository.ModelMetaNames.AUTHOR_NAME: 'Your Name'
}

In [ ]:
# store the (scikit-learn) model in WML
stored_model = wml_client.repository.store_model(reg, meta_props=metadata)

In [ ]:
# get the id of the stored model
published_model_uid = wml_client.repository.get_model_uid(stored_model)

In [ ]:
# create deployment and fetch scoring endpoint
created_deployment = wml_client.deployments.create(published_model_uid, name="Housing Prices Deployment")
scoring_endpoint = wml_client.deployments.get_scoring_url(created_deployment)

## Deployment validation

Use the stored deployment to make a prediction.

In [ ]:
# review test data
X_test[0:2]

In [ ]:
y_test[0:2]

In [ ]:
# create scoring payload
scoring_values = X_test[0:2].tolist()
scoring_payload = {"values": scoring_values}
print(scoring_payload)

In [ ]:
# run prediction
predictions = wml_client.deployments.score(scoring_endpoint, scoring_payload)
print(predictions)

Do the results match your expectation? Are the estimations accurate?

In [ ]:
# use the local model to make the same prediction in your notebook and compare the results
reg.predict(X_test[0:2])